<a href="https://colab.research.google.com/github/emmacwatts/AutoTagsCRISPR/blob/main/utils_2022_12_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def check_start_stop_NGG(df):
    '''
    df: dictionary
            df={
            "start/stop":"start",#is it N or C termini -> do we need to look at start or stop codon for teh cut 
            'genome_start_codon_pos':400, 
            'genome_stop_codon_pos':700, #or only 1 of those 
            'strand_type':'+',
            "sgRNA_list_pos":[[401,425],[456,467],[478,489],[395,415]],#those wil be as genome positions -assumptions - the coordinates correspond to the 1st and last bp of the strand to which the gsRNA will be complementary to
            "sgRNA_list_values":["AAGCGACTA","AAAAAAAATAAAAA","ATATATTTTTTTTTTAAAAA","AGCGCGAAATAATA"]
        }
    '''
    for i in range(0,len(df["sgRNA_list_values"])):
        cand=df["sgRNA_list_values"][i]#canditate sgRNA
        cand_len=len(cand) #canditates lenght 
        df["best_gsRNA"]=[]
        
        if df['strand_type']=="+":
            if df["start/stop"]=="start_codon":
                if cand[0:4]=="ATGG": #checking if the PAM (last 3bp of sgRNA) include a ATG- its NGG so only way it can include the G from ATG is if it is TGG or GGG
                    df["best_gsRNA"]=df["sgRNA_list_values"][i]
            if df["start/stop"]=="stop_codon":
                if cand[(cand_len-4):cand_len]=="TAGG":
                    df["best_gsRNA"]=df["sgRNA_list_values"][i]
        if df['strand_type']=="-":
            if df["start/stop"]=="start_codon":
                if cand[(cand_len-4):cand_len]=="TACC":
                    df["best_gsRNA"]=df["sgRNA_list_values"][i]
            if df["start/stop"]=="stop_codon":
                if cand[0:4]=="ATCC":
                    df["best_gsRNA"]=df["sgRNA_list_values"][i]
        out=df["best_gsRNA"]
        return out

In [ ]:
def exclude_over15(df):

In [ ]:
  stop_start_pos = df["genome_start_codon_pos"]
  list_of_ok=[]
  list_of_ok_pos=[]
  for i in range(0, len(df["sgRNA_list_values"])):
    if abs(df["sgRNA_list_pos"][i][0]-stop_start_pos)<16 and abs(df["sgRNA_list_pos"][i][1]-stop_start_pos)<16:
      list_of_ok.append(df["sgRNA_list_values"][i])
      list_of_ok_pos.append(df["sgRNA_list_pos"][i])
  df["sgRNA_list_values"]=list_of_ok
  df["sgRNA_list_pos"]=list_of_ok_pos
  return df["sgRNA_list_values"],df['sgRNA_list_pos']

In [ ]:
def select_closest(df):
  list_of_dist=len(df["sgRNA_list_values"])*[""]
  list_of_val=len(df["sgRNA_list_values"])*[""]
  if df["strand_type"]=="+":
    for i in range(0,len(df["sgRNA_list_values"])):
        cutting_pos=(df["sgRNA_list_pos"][i][0]+6)
        if df["start/stop"]=="start_codon":
           start_stop_pos=df["genome_start_codon_pos"]
        else:
          start_stop_pos=df["genome_stop_codon_pos"]
        dist=abs((cutting_pos-start_stop_pos))
        list_of_dist[i]=dist
        list_of_val[i]=df["sgRNA_list_values"][i]
  else:
    cutting_pos=(df["sgRNA_list_pos"][i][1]-6)
    for i in range(0,len(df["sgRNA_list_values"])):
        if df["start/stop"]=="start_codon":
           start_stop_pos=df["genome_start_codon_pos"]
        else:
            start_stop_pos=df["genome_stop_codon_pos"]
        dist=abs((cutting_pos-start_stop_pos)) 
        list_of_dist[i]=dist
        list_of_val[i]=df["sgRNA_list_values"][i]
  smallest=min(list_of_dist)
  for i in range(0,len(list_of_dist)):
    if list_of_dist[i]==smallest:
      best_sgRNA=list_of_val[i]

In [ ]:
  return best_sgRNA

In [ ]:
def find_best_gRNA(df):
  best_sgRNA=check_start_stop_NGG(df)
  
  if best_sgRNA!=[]:
    return best_sgRNA
  else:
    if df["start/stop"]=="start_codon":
      exclude_over15(df)
    else:
      exclude_over15(df) 
    if len(df['sgRNA_list_values'])!=0: #checking that any canditates met the condition
      best_sgRNA=select_closest()
  #if best_sgRNA!=[]:
    #get a new batch of sgRNA's and reapet process
  return best_sgRNA

In [ ]:
def filter_gRNA(gRNA_file, TF_dict):
    '''
    gRNA_file: gff file
    TF_dict: dictionary
    '''
    import pandas as pd
    

    # creating a data frame from the gRNA files
    # makes pandas dataframe from csv file
    gRNAFileAnnotation = pd.read_csv(gRNA_file, sep = "\t", index_col = False)

    #This is to reformat the "Attribute" category in refGenomeAnnotation, to extract Gene_ID, Gene_Symbol, and Transcript ID
    #index = 0

    #Add new categories to the dataframe
    gRNAFileAnnotation = gRNAFileAnnotation.assign(target_site_variation= "")

    #This is to reformat the "Attribute" category in refGenomeAnnotation, to extract Gene_ID, Gene_Symbol, and Transcript ID
    index = 0

    #For each attribute value, extract the gene ID and symbol and add this to the new categories
    for attribute in gRNAFileAnnotation['attributes']:
        fullatt = (gRNAFileAnnotation.loc[index]["attributes"]).split(";")
        gRNAFileAnnotation.at[index,"target_site_variation"] = fullatt[8]
        index+=1
    
    # shorten file to essential information
    GenomeCoordinates = gRNAFileAnnotation.loc[:,["target_site_variation", "fmin", "fmax"]]

    #filter out Start and Stop sequences within TF gene
    num_gRNA_matches = 1

    #initialize a list that stores the positions of the gRNAs
    gRNA_list_pos = []
    
    for gRNA in range(len(GenomeCoordinates)):
        
        if int(GenomeCoordinates['fmin'].iloc[gRNA]) >= TF_dict["Start"] - 20 and int(GenomeCoordinates['fmax'].iloc[gRNA]) <= TF_dict["Stop"] + 20 \
            and GenomeCoordinates['target_site_variation'].iloc[gRNA] == "target_site_variation=none observed":
            
            gRNA_list_pos.append([int(GenomeCoordinates['fmin'].iloc[gRNA]), int(GenomeCoordinates['fmax'].iloc[gRNA])])
            num_gRNA_matches = num_gRNA_matches + 1
        
    TF_dict["sgRNA_list_pos"] = gRNA_list_pos
    return(TF_dict)

In [ ]:
def make_list_of_dict_from_df(TF_genes_list):
    import pandas as pd
    TF_list_of_dict = []
    TFsdf = pd.read_csv(TF_genes_list, sep = "\t", index_col = False)
    TFsdict = {}
    for index, rowcontents in TFsdf.iterrows():
        TFsdict[index] = rowcontents.to_dict()
    print(TFsdict)

In [ ]:
def does_log_exist(TF_list, ref_genome, annotation, transgenic_genome_chr2, transgenic_genome_chr3):
    from os.path import exists
    import pandas as pd
    if exists("TF_log_mock.csv"):
        print("logfile exists")
        
    else:
        print("logfile does not yet exist ... creating log file")
        TFsdf = make_dataframe_from_TFs_list(TF_list, ref_genome, annotation, transgenic_genome_chr2, transgenic_genome_chr3)
    
    return

In [ ]:
def revComp(inputSeq):
  """
  This function takes an input sequence and returns the reverse complement.
  
  Input: inputSeq in str format
  Output: revComp in str format
  """
  
  complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}

  revComp = ""
  for base in inputSeq[::-1]:
    revComp += complement[(base.upper())]
  return revComp

In [ ]:
def make_dataframe_from_TFs_list(TF_list, ref_genome, annotation, transgenic_genome_chr2, transgenic_genome_chr3):
    '''
    input
    output
    '''

    import pandas as pd
    from Bio import SeqIO

    #This is the input file containing the TFs we want to query
    #Imported as a pandas dataframe
    queryTFsdf = pd.read_excel(TF_list)

    #This is the .gtf file with annotations for each gene on the reference genome
    refAnnotationsHeaders = ["Chromosome", "Source", "Gene_Region", "Start", "Stop", "Score", "Strand", "Frame", "Attribute"]
    refGenomeAnnotation = pd.read_csv(annotation, sep = "\t", header = None, index_col = False, names = refAnnotationsHeaders)

    #This is the FASTA file of the reference genome sequence
    refSeqPerChromosome = {}
    for seq in SeqIO.parse(open(ref_genome), 'fasta'):
        refSeqPerChromosome[seq.id] = seq.seq
    
    #This is to reformat the "Attribute" category in refGenomeAnnotation, to extract Gene_ID, Gene_Symbol, and Transcript ID
    index = 0

    #Add new categories to the dataframe
    refGenomeAnnotation = refGenomeAnnotation.assign(Gene_ID = "", Gene_Symbol = "", Transcript_ID = "")

    #For each attribute value, extract the gene ID and symbol and add this to the new categories
    for attribute in refGenomeAnnotation['Attribute']:
        fullatt = (refGenomeAnnotation.loc[index]["Attribute"]).replace(";", "")
        fullatt = fullatt.replace('"', "")
        fullattsplit = fullatt.split(" ")
        refGenomeAnnotation.at[index,"Gene_ID"] = fullattsplit[1]
        refGenomeAnnotation.at[index,"Gene_Symbol"] = fullattsplit[3]
        if len(fullattsplit) == 8:
            refGenomeAnnotation.at[index,"Transcript_ID"] = fullattsplit[5]
        index+=1

    #Delete Attributes category
    del refGenomeAnnotation["Attribute"]

    #Select only rows that TFs are in, and keep only the start and stop codon gene regions

    refGenomeAnnotation = refGenomeAnnotation.loc[refGenomeAnnotation["Gene_Region"].isin(["start_codon", "stop_codon"])]

    TFsdf = refGenomeAnnotation[["Gene_ID", "Transcript_ID", "Chromosome", "Gene_Region", "Start", "Stop", "Strand"]].loc[refGenomeAnnotation["Gene_ID"].isin(queryTFsdf["Flybase_ID"])]

    #Add reference genome sequence per gene region
    #This will correspond to 1.3kb upstream and downstream of ATG/stop codon 
    TFsdf = TFsdf.assign(Reference_Seq = "", Transgenic_Seq = "")

    for index, rowcontents in TFsdf.iterrows():
        if rowcontents["Strand"] == "+":

            #Define 2.6kb gene region
            regionStart = rowcontents["Start"] - 1701
            regionStop = rowcontents["Stop"] + 1700

            #Add reference sequence
            TFsdf.at[index,"Reference_Seq"] = str(refSeqPerChromosome[rowcontents["Chromosome"]][regionStart:regionStop])

        if rowcontents["Strand"] == "-":

            #Define 2.6kb gene region
            regionStart = rowcontents["Start"] - 1701
            regionStop = rowcontents["Stop"] + 1700

            #Add reference sequence
            refPosStrandSeq = str(refSeqPerChromosome[rowcontents["Chromosome"]][regionStart:regionStop]) #This is the + strand seq, so goes from end to beginning
            TFsdf.at[index,"Reference_Seq"] = revComp(refPosStrandSeq)

    TFsdict_of_dict = {}
    for index, rowcontents in TFsdf.iterrows():
        TFsdict_of_dict[index] = rowcontents.to_dict()

    return TFsdf, TFsdict_of_dict